# Text Cleaning Pipeline
The goal of this file is to import the comments and news sources, join the data, and find the sentiment scores of both the source and the cleaned text before performing analysis.

In [1]:
from _functions import *

c:\Users\Trevo\OneDrive\Documents\GitHub\NLP-Midterm\Code\_functions.py:14: FutureWarning: The demoji.download_codes attribute is deprecated and will be removed from demoji in a future version. It is an unused attribute as emoji codes are now distributed directly with the demoji package.
  demoji.download_codes()
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Trevo\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Trevo\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


## Create Model Input
Complete this process through concatenating the comments and the video transcripts

In [2]:
parent_directory = Path().resolve().parent
data_directory = parent_directory / "Data"
comments_directory = data_directory / "video_comments"
transcripts_directory = data_directory / "video_transcripts"

In [3]:
DUDNUM = 999999999

reliability_scores = {"nbc_news": 42.78, "wsj": 48.87, "bbc_news": 44.73, "bloomberg" : 42.11,
                      "cnn" : 42.13, "sixty_minutes" : 34.19, "sky_news" : 42.24,
                       "wusa": 46.67, "dw_news" : DUDNUM, "forbes" : 41.06,
                        "fox_news" : 35.83, "podcast_cvt" : DUDNUM, "econ_explained": DUDNUM,
                        "ny_post": 31.82}

bias_scores = {"nbc_news": -5.61, "wsj": 4.50, "bbc_news" : -1.33, "bloomberg": -3.16, "cnn" : -6.18,
               "sixty_minutes" : -9.55, "sky_news" : -0.88, "wusa": -1.50, "dw_news" : DUDNUM,
               "forbes": -2.88, "fox_news": 11.06, "podcast_cvt" : DUDNUM, "econ_explained": DUDNUM,
               "ny_post": 9.33}

bias_ratings = {"nbc_news": "Left", "wsj": "Right", "bbc_news" : "Center", "bloomberg": "Left", "cnn" : "Left",
               "sixty_minutes" : "Left", "sky_news" : "Center", "wusa": "Center", "dw_news" : DUDNUM,
               "forbes": "Center", "fox_news": "Right", "podcast_cvt" : DUDNUM, "econ_explained": DUDNUM, "ny_post": "Right"}

## Data Processing and Sentiment Analysis

This code processes video comments and transcripts, extracting relevant information and calculating sentiment scores. It performs the following steps:

1. **Initialize DataFrames**: Creates empty DataFrames for storing transcripts and comments.
2. **Iterate Over Files**: Loops through each file in the comments directory.
3. **Read Comments**: Reads the comments from CSV files, dropping any rows with missing values and unnecessary columns.
4. **Read Transcripts**: Reads the corresponding transcript files if they exist.
5. **Extract Information**: Extracts the source name and index from the file name.
6. **Filter Data**: Filters out sources with invalid reliability scores.
7. **Add Metadata**: Adds metadata such as source name, reliability score, and bias score to the comments DataFrame.
8. **Sentiment Analysis**: Calculates VADER sentiment scores for both the transcript and comments.
9. **Append Data**: Appends the processed data to the respective DataFrames.
10. **Save Data**: Saves the final DataFrames to CSV files.

In [4]:
transcript_df = pd.DataFrame(columns=['index', 'source', 'transcript'])

comments_df = pd.DataFrame(columns=['index', 'source', 'leaning', 'reliability_score', 'bias_score', 'vader_comment', 'comment'])

# Iterate over each .txt file in the directory
for file_name in os.listdir(comments_directory):

    # Construct comments file path
    file_path = comments_directory / file_name

    # Remove ".csv" and concatenate "_transcript.txt" for file_name
    transcript_file_name = file_name.replace(".csv", "_transcript.txt")
    transcript_file_path = transcripts_directory / transcript_file_name

    # TODO : Read in the CSV file for the comments
    input_df = pd.read_csv(file_path, encoding="utf-8")
    input_df = input_df.dropna()

    # Drop 'Unnamed: 0' column if it exists, as not all csv pulls have the index
    if 'Unnamed: 0' in input_df.columns:
        input_df = input_df.drop(columns=['Unnamed: 0'])

    # TODO : Read in the transcript file and append score to dataframe
    # NOTE : Cannot make use of any data without a transcript
    if transcript_file_path.exists():
        with open(transcript_file_path, "r", encoding="utf-8") as file:
            transcript_text = file.read().strip()

        # DONE : Define unique identifiers for the data
        source_name = re.sub(r"[0-9]+\.csv", "", file_name)
        index = int(re.sub(r"\D", "", file_name))

        # TODO : Turn this into a function
        if reliability_scores[source_name] != DUDNUM:

            # TODO : Add the news source name to the dataframe
            input_df['index'] = index
            input_df["source"] = source_name
            input_df['leaning'] = bias_ratings[source_name]
            input_df["reliability_score"] = reliability_scores[source_name]
            input_df["bias_score"] = bias_scores[source_name]

            # TODO : Add the VADER sentiment score to the dataframe
            input_df["vader_transcript"] = analyze_sentiment_vader(transcript_text)

            # TODO : Clean the comment
            input_df["clean_comment"] = input_df.apply(lambda row: preprocess_text(row['comment']), axis=1)

            input_df["vader_comment"] = input_df.apply(lambda row: analyze_sentiment_vader(row['clean_comment']), axis=1)

            # DONE : Append the transcript as a row to the transcript dataframe
            np_array_entry = np.array([index, source_name, transcript_text])
            transcript_df.loc[len(transcript_df)] = np_array_entry


            # Drop the 0th index row
            input_df = input_df.drop(0)

            comments_df = pd.concat([comments_df, input_df])

C:\Users\Trevo\AppData\Local\Temp\ipykernel_25380\2758356490.py:59: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.


## Determine the sentiment of the video
VADER shall be used in this section to determine the sentiment of the videos in question

In [6]:
# TODO : Iterate through the transcripts and determine the sentiment of each
transcript_df['sentiment'] = transcript_df.apply(lambda row: analyze_sentiment_vader(row['transcript']), axis=1)
transcript_df.head()

,index,source,transcript,sentiment
0,1,bbc_news,We begin with the breaking news. A fresh trade...,"{'compound': -0.8748, 'positive': 0.087, 'neut..."
1,2,bbc_news,There are growing fears of a major global trad...,"{'compound': 0.825, 'positive': 0.095, 'neutra..."
2,3,bbc_news,I'm Helena Humphrey. Good to have you with us....,"{'compound': 0.9803, 'positive': 0.092, 'neutr..."
3,1,bloomberg,"Congratulations, Mr.\n\nWell, he's back and gu...","{'compound': -0.9962, 'positive': 0.076, 'neut..."
4,3,bloomberg,We are continuing to get headlines out of the ...,"{'compound': 0.9854, 'positive': 0.067, 'neutr..."


## Determining Sarcasm of the Comment

In [6]:
import pandas as pd
from _functions import get_relative_path

sarcasm_df = pd.read_csv(get_relative_path() / "reddit_sarcasm_training.csv", encoding="utf-8")
sarcasm_df.head()

,label,comment
0,0,NC and NH.
1,0,I think a significant amount would be against ...
2,0,because it's what really bothers him... and it...
3,0,Conservatism as an ideology is for sure a reac...
4,0,"Maybe not control, but certainly that is evide..."


In [7]:
# Drop NaN values
sarcasm_df = sarcasm_df.dropna()
sarcasm_df = sarcasm_df.reset_index(drop=True)
sarcasm_df.head()

,label,comment
0,0,NC and NH.
1,0,I think a significant amount would be against ...
2,0,because it's what really bothers him... and it...
3,0,Conservatism as an ideology is for sure a reac...
4,0,"Maybe not control, but certainly that is evide..."


In [8]:
sarcasm_sample_df = sarcasm_df.sample(n=2000, random_state=8)
sarcasm_sample_df = sarcasm_sample_df.reset_index(drop=True)
sarcasm_sample_df.head()

,label,comment
0,1,But the problem is how progressives have faile...
1,0,"I'll vote against Hillary, thanks tho!"
2,1,As opposed to all of the other qualified and c...
3,0,Dribble.
4,0,"By forming a PAC, money that is donated and no..."


In [9]:
from BERT import SarcasmDetector
import pandas as pd

detector = SarcasmDetector()
detector.train(sarcasm_sample_df['comment'].tolist(), sarcasm_sample_df['label'].tolist())


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1, Loss: 0.6346621324121952
Epoch 2, Loss: 0.4614064587652683
Epoch 3, Loss: 0.2269466827530414
Validation Accuracy: 0.7125
              precision    recall  f1-score   support

           0       0.66      0.57      0.62       160
           1       0.74      0.80      0.77       240

    accuracy                           0.71       400
   macro avg       0.70      0.69      0.69       400
weighted avg       0.71      0.71      0.71       400

['Sarcastic', 'Not Sarcastic', 'Sarcastic']


In [10]:
# Test sarcasm detection
test_text = sarcasm_sample_df["comment"].iloc[8]
print(f"Text: {test_text}\nSarcastic? {detector.predict([test_text])}")

Text: yeah but that was in 2012 dollars.
Sarcastic? ['Sarcastic']


In [11]:
comments_df['sarcasm'] = detector.predict(comments_df['clean_comment'].tolist())

In [12]:
# Define a function to categorize VADER sentiment
def categorize_sentiment(vader_score):
    if vader_score['compound'] >= 0.05:
        return 'positive'
    elif vader_score['compound'] <= -0.05:
        return 'negative'
    else:
        return 'neutral'

# Apply the function to create a new column for sentiment category
comments_df['sentiment_category'] = comments_df['vader_comment'].apply(categorize_sentiment)

# Create a pivot table to count sarcasm occurrences by sentiment category
sarcasm_table = comments_df.pivot_table(index='sentiment_category', columns='sarcasm', aggfunc='size', fill_value=0)

print(sarcasm_table)

sarcasm             Not Sarcastic  Sarcastic
sentiment_category                          
negative                      870       2047
neutral                      1038       1596
positive                      954       2567


In [7]:
# Save data for analysis

comments_df.to_csv(get_relative_path() / "comments.csv", index=False)
transcript_df.to_csv(get_relative_path() / "transcripts.csv", index=False)